# Dados

## Paths

In [ ]:
datasets_path     = 'datasets/annotated/'
models_path       = 'results/models/'
tensorboard_path  = 'results/Tensorboard/YOLO/'

## Dataloader

In [ ]:
from torch.utils.data import DataLoader
import torchvision

import matplotlib.pyplot as plt
import numpy as np

def my_imshow(img, dataset, numImages=10):

    if dataset == 'cifar10' :
        img = img / 2 + 0.5     # unnormalize

    img = torchvision.utils.make_grid(img[:numImages],nrow=numImages//2)

    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))

    plt.axis('off')
    plt.imshow(npimg)
    plt.show()

def show_images(train_loader, test_loader, dataset, numImages=10) :
    print('Train samples')
    # get some random training images
    dataiter = iter(train_loader)
    images = next(dataiter)[0]
    my_imshow(images, dataset, numImages)

    print('Test samples')
    # get some random training images
    dataiter = iter(test_loader)
    images = next(dataiter)[0]
    my_imshow(images, dataset, numImages)

def get_data_generic ( batch_size , show_image=False, numImages=8 ) :
    train_dataset = f'{datasets_path}/train/'
    test_dataset = f'{datasets_path}/test/'
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    if show_image :
        show_images(train_loader, test_loader, 'generic_gest', numImages)
    
    return train_loader, test_loader, len(train_dataset)

def get_data_my ( batch_size , show_image=False, numImages=8 ) :
    train_dataset = f'{datasets_path}/train/'
    test_dataset = f'{datasets_path}/test/'
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    if show_image :
        show_images(train_loader, test_loader, 'my_gest', numImages)

    return train_loader, test_loader, len(train_dataset)

In [ ]:
get_data_my(batch_size=256, show_image=True, numImages=16);

# Rede

## Arquitetura

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F



ModuleNotFoundError: No module named 'torch'